# Optimization Model for MSBA Student Practicum Assignment

Built by Younjoo Lee with Co-pilot ChatGPT

## Set Up

In [ ]:
%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

from pyomo.environ import *

In [ ]:
import pandas as pd
import random

## Model Building
I used ChatGPT to create a base model and help me build a foundation.
[Here's the link to the prompts.](https://chat.openai.com/share/59834fa0-7715-4270-adbd-a660ee1d53c2). However, it is critical to understand what the model is doing; hence, I've written comments and provided text of my understanding for each line of code.


In [ ]:
# Set random seed for reproducibility - I don't want the random numbers to change each time I run the code.
# 42 was a random number I chose
random.seed(42)

### Customizable Inputs
Users can change the number of students, projects, industries, team size, and top industries which is the "each student must get their top _ industry choice."

In [ ]:
# Number of students, projects, and industries
num_students = 110
num_projects = 18
num_industries = 12
team_size = 7
top_industries_count = 5

I am working with random number rankings for each 110 student; however, users can input a list of the industry rankings for each student as well. The code below generates a list of student rankings which are, again, random number generations from 1 - 12.

In [ ]:
# Generate random industry rankings for each student
student_rankings = {i: random.sample(range(1, num_industries + 1), num_industries) for i in range(1, num_students + 1)}

# Convert rankings to list format
student_rankings_list = {i: [student_rankings[i][k-1] for k in range(1, num_industries + 1)] for i in range(1, num_students + 1)}

### Create Model

In [ ]:
# Create a concrete model
model = ConcreteModel()

The binary decision or "yes/no" variables include the projects 1-18 and the industries 1-12.

In [ ]:
# Define binary decision variables
model.x = Var(range(1, num_students + 1), range(1, num_projects + 1), range(1, num_industries + 1), within=Binary)
model.d = Var(range(1, num_industries + 1), within=NonNegativeReals)

This line of code is creating a variable called "abs_diff" which measures the absolute differences between the rankings that students prefer and the projects they are actually assigned to.

In [ ]:
# Define auxiliary variables for absolute differences
model.abs_diff = Var(range(1, num_students + 1), within=NonNegativeReals)

We are defining the objective which is to minimize the difference between the absolute differences in student preference in ranking and their actual assignment.

In [ ]:
# Define objective function to minimize the absolute differences
model.obj = Objective(expr=sum(model.abs_diff[k] for k in range(1, num_industries + 1)), sense=minimize)

Constraints:

*   Each student must be assigned to 1 project in 1 industry
*   There must be up to <= 7 students per project
*   Each student must be assigned to one of their top 5 industry choices
*   Calculating the absolute differences between the actual number of students assigned to a project ina specific industry and the desired team size.

In [ ]:
# Define constraints, every student must be 1 project in 1 industry
model.assignment_constraint = ConstraintList()
for i in range(1, num_students + 1):
    model.assignment_constraint.add(sum(model.x[i, j, k] for j in range(1, num_projects + 1) for k in range(1, num_industries + 1)) == 1)

# Define constraints for absolute differences
model.abs_diff_constraint = ConstraintList()
for k in range(1, num_industries + 1):
    model.abs_diff_constraint.add(model.abs_diff[k] == sum(model.x[i, j, k] for i in range(1, num_students + 1) for j in range(1, num_projects + 1)) - team_size)

# Constraint to ensure each student is assigned to one of their top 5 industries
model.top_industries_constraint = ConstraintList()
for i in range(1, num_students + 1):
    top_industries = sorted(student_rankings[i][:top_industries_count])
    model.top_industries_constraint.add(sum(model.x[i, j, k] for j in range(1, num_projects + 1) for k in top_industries) >= 1)

# Constraint to limit each project to a maximum of 7 students
model.project_limit_constraint = ConstraintList()
for j in range(1, num_projects + 1):
    model.project_limit_constraint.add(sum(model.x[i, j, k] for i in range(1, num_students + 1) for k in range(1, num_industries + 1)) <= team_size)

Solve the optimization problem automatically through programs.

In [ ]:
# Solve the optimization problem
solver = SolverFactory('cbc')
solver.solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 26.0, 'Upper bound': 26.0, 'Number of objectives': 1, 'Number of constraints': 250, 'Number of variables': 23772, 'Number of binary variables': 23760, 'Number of integer variables': 23760, 'Number of nonzeros': 12, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 2.42, 'Wallclock time': 2.77, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}, 'Black box': {'Number of iterations': 0}}, 'Error rc': 0, 'Time': 2.8639354705810547}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

### Model Outcome/Output



In [ ]:
# Display the results
for i in range(1, num_students + 1):
    for j in range(1, num_projects + 1):
        for k in range(1, num_industries + 1):
            if value(model.x[i, j, k]) > 0.5:
                print(f"Student {i} is assigned to Project {j} in Industry {k} (Ranking: {student_rankings_list[i]})")

# Display the optimal objective value (minimized absolute differences)
print("Optimal objective value:", model.obj())

# Display the number of students in each project
project_sizes = {j: sum(sum(value(model.x[i, j, k]) for k in range(1, num_industries + 1)) for i in range(1, num_students + 1)) for j in range(1, num_projects + 1)}

print("\nNumber of students in each project:")
for j, size in project_sizes.items():
    print(f"Project {j}: {size} students")


Student 1 is assigned to Project 2 in Industry 1 (Ranking: [11, 2, 1, 5, 4, 12, 7, 10, 9, 3, 6, 8])
Student 2 is assigned to Project 3 in Industry 10 (Ranking: [1, 2, 4, 10, 12, 5, 11, 7, 9, 8, 6, 3])
Student 3 is assigned to Project 15 in Industry 1 (Ranking: [1, 3, 7, 6, 5, 2, 10, 11, 12, 4, 9, 8])
Student 4 is assigned to Project 16 in Industry 12 (Ranking: [6, 12, 10, 5, 1, 11, 4, 9, 8, 2, 7, 3])
Student 5 is assigned to Project 10 in Industry 11 (Ranking: [11, 10, 6, 4, 2, 1, 12, 8, 3, 7, 9, 5])
Student 6 is assigned to Project 4 in Industry 3 (Ranking: [7, 5, 8, 6, 3, 10, 12, 2, 9, 4, 1, 11])
Student 7 is assigned to Project 8 in Industry 8 (Ranking: [9, 4, 3, 8, 7, 10, 6, 5, 2, 12, 11, 1])
Student 8 is assigned to Project 7 in Industry 7 (Ranking: [4, 1, 6, 7, 5, 11, 2, 8, 3, 9, 10, 12])
Student 9 is assigned to Project 13 in Industry 11 (Ranking: [11, 8, 3, 5, 10, 2, 6, 9, 12, 4, 7, 1])
Student 10 is assigned to Project 9 in Industry 9 (Ranking: [6, 4, 3, 9, 8, 1, 7, 12, 2, 10,